In [1]:
from prompting.llms.utils import GPUInfo
GPUInfo.total_memory, GPUInfo.free_memory, GPUInfo.used_memory

2024-08-14 12:20:24.357 | INFO     | prompting:<module>:38 - Prompting version: 2.7.0
2024-08-14 12:20:27.393 | INFO     | prompting.utils.config:config:27 - RUNNING WITH ARGS: netuid=None wallet.name=None wallet.hotkey=None subtensor.network=None axon.port=None
2024-08-14 12:20:27.397 | INFO     | prompting.settings:<module>:12 - Config: 
netuid: null
wallet:
  name: null
  hotkey: null
subtensor:
  network: null
axon:
  port: null
no_prompt: false
config: null
strict: false
no_version_checking: false

2024-08-14 12:20:27.516 | INFO     | prompting.utils.config:config:27 - RUNNING WITH ARGS: netuid=None wallet.name=None wallet.hotkey=None subtensor.network=None axon.port=None
2024-08-14 12:20:27.518 | INFO     | prompting.settings:load_env:121 - Instantiating bittensor objects with NETUID: 61, WALLET_NAME: validator, HOTKEY: validator_hotkey
2024-08-14 12:20:31.125 | INFO     | prompting.settings:load_env:130 - Bittensor objects instantiated... WALLET: wallet(validator, validator_hotk

mode='validator' MOCK=False NO_BACKGROUND_THREAD=True WANDB_ON=True WANDB_ENTITY='felix-quinque-macrocosmos-ai' WANDB_PROJECT_NAME='validator' WANDB_RUN_STEP_LENGTH=100 WANDB_API_KEY='ae29a588c238d0e168d620e0b18a5e29e283935a' WANDB_OFFLINE=False WANDB_NOTES='' SAVE_PATH='./storage' NEURON_EPOCH_LENGTH=1 NEURON_DEVICE='cuda' NEURON_GPUS=1 LOGGING_DONT_SAVE_EVENTS=False NEURON_TIMEOUT=15 NEURON_DISABLE_SET_WEIGHTS=False NEURON_MOVING_AVERAGE_ALPHA=0.1 NEURON_DECAY_ALPHA=0.001 NEURON_AXON_OFF=False NEURON_VPERMIT_TAO_LIMIT=4096 NEURON_QUERY_UNIQUE_COLDKEYS=False NEURON_QUERY_UNIQUE_IPS=False NEURON_FORWARD_MAX_TIME=120 ORGANIC_TIMEOUT=15 ORGANIC_SAMPLE_SIZE=10 ORGANIC_REUSE_RESPONSE_DISABLED=False ORGANIC_REFERENCE_MAX_TOKENS=256 ORGANIC_SYNTH_REWARD_SCALE=1.0 ORGANIC_SET_WEIGHTS_ENABLED=True ORGANIC_DISABLED=False ORGANIC_TRIGGER_FREQUENCY=120 ORGANIC_TRIGGER_FREQUENCY_MIN=5 ORGANIC_TRIGGER='seconds' ORGANIC_SCALING_FACTOR=1 LOG_FULL=False NETUID=61 TEST=True OPENAI_API_KEY='sk-proj-Cq96

(44.3516845703125, 44.089599609375, 0.2620849609375)

In [2]:
from typing import ClassVar
from loguru import logger
import numpy as np
from pydantic import BaseModel, ConfigDict, model_validator
import torch
import vllm


import vllm
import numpy as np
from prompting.llms.utils import GPUInfo
from vllm.distributed.parallel_state import destroy_model_parallel
from prompting.settings import settings


class ModelConfig(BaseModel):
    model_id: str
    reward: float
    min_ram: float
    model_config = ConfigDict(frozen=True)

    def __hash__(self):
        return hash((self.model_id, self.reward, self.min_ram))


class ModelZoo:
    models_configs: ClassVar[list[ModelConfig]] = [
        ModelConfig(model_id="casperhansen/mistral-nemo-instruct-2407-awq", reward=0.1, min_ram=24),
        ModelConfig(model_id="casperhansen/llama-3-8b-instruct-awq", reward=0.1, min_ram=24),
        ModelConfig(model_id="casperhansen/llama-3-70b-instruct-awq", reward=0.8, min_ram=70),
    ]

    @classmethod
    def get_all_models(cls) -> list[str]:
        return [model.model_id for model in cls.models_configs]

    @classmethod
    def get_random(cls, max_ram: float = np.inf) -> ModelConfig:
        models = [model for model in cls.models_configs if model.min_ram <= max_ram]
        return np.random.choice(models)

    @classmethod
    def get_model_by_id(cls, model_id: str) -> ModelConfig:
        return [model for model in cls.models_configs if model.model_id == model_id][0]


class ModelManager(BaseModel):
    always_active_models: list[ModelConfig] = []
    total_ram: float = 40.0
    active_models: dict[ModelConfig, vllm.LLM] = {}
    used_ram: float = 0.0
    model_config = ConfigDict(arbitrary_types_allowed=True)

    @model_validator(mode="after")
    def load_always_active_models(self) -> "ModelManager":
        for model_config in self.always_active_models:
            self.load_model(model_config)
        return self

    def load_model(self, model_config: ModelConfig, force: bool = True):
        # if force loading is enabled, unload models until there is enough RAM
        if force:
            for active_model in self.active_models.keys():
                if active_model in self.always_active_models:
                    continue
                if self.used_ram + model_config.min_ram > self.total_ram or GPUInfo.free_memory < model_config.min_ram:
                    logger.debug(f"Unloading {active_model.model_id} to make room for {model_config.model_id}")
                    self.unload_model(active_model)
                else:
                    break

        if self.used_ram + model_config.min_ram > self.total_ram or GPUInfo.free_memory < model_config.min_ram:
            raise MemoryError(
                f"""Not enough RAM to load model {model_config.model_id}. 
                    Required: {model_config.min_ram} GB
                    Available in Model Manager: {self.total_ram - self.used_ram} GB
                    Available in GPU: {GPUInfo.free_memory} GB"""
            )

        if model_config in self.active_models.keys():
            print(f"Model {model_config.model_id} is already loaded.")
            return

        try:
            model = vllm.LLM(model_config.model_id, max_model_len=8_000)
            self.active_models[model_config] = model
            self.used_ram += model_config.min_ram
            logger.info(f"Model {model_config.model_id} loaded. Current used RAM: {self.used_ram} GB")

            return model
        except Exception as e:
            logger.exception(f"Failed to load model {model_config.model_id}. Error: {str(e)}")

    def unload_model(self, model_config: ModelConfig):
        if model_config not in self.active_models:
            logger.warning("Couldn't find model to unload.")
            return
        import gc

        destroy_model_parallel()
        try:
            del self.active_models[model_config].llm_engine.model_executor.driver_worker
            del self.active_models[model_config]
        except:
            pass
        gc.collect()
        self.used_ram -= model_config.min_ram
        torch.cuda.empty_cache()

    def get_or_load(self, model_id: str) -> vllm.LLM:
        model_config = ModelZoo.get_model_by_id(model_id)
        if model_config not in self.active_models:
            self.load_model(model_config)
        return self.active_models[model_config]


# keep model used for validation always active


import asyncio
import random
from pydantic import BaseModel
from loguru import logger

class AsyncModelScheduler(BaseModel):
    model_manager: ModelManager
    interval: int = 10  # Minimum time in seconds for a model to stay active
    running: bool = False

    async def start(self):
        self.running = True
        await self.run_scheduler()

    async def stop(self):
        self.running = False

    async def run_scheduler(self):
        while self.running:
            selected_model = ModelZoo.get_random(max_ram=self.model_manager.total_ram)
            logger.info(f"Loading model {selected_model.model_id} for {self.interval} seconds.")
            
            if selected_model in self.model_manager.active_models:
                logger.info(f"Model {selected_model.model_id} is already loaded.")
                return
            # Load the selected model
            await self.load_model_async(selected_model)
            
            # Keep the model loaded for the specified time interval
            await asyncio.sleep(self.interval)
            
            # After the interval, unload the model if it is not in always_active_models
            if selected_model not in self.model_manager.always_active_models:
                logger.info(f"Unloading model {selected_model.model_id} after {self.interval} seconds.")
                self.model_manager.unload_model(selected_model)

        logger.info("Model scheduler stopped.")

    async def load_model_async(self, model_config: ModelConfig):
        loop = asyncio.get_event_loop()
        await loop.run_in_executor(None, self.model_manager.load_model, model_config)


/root/.cache/pypoetry/virtualenvs/prompting-fb5sw-i7-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-08-14 12:20:32,206	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
/root/.cache/pypoetry/virtualenvs/prompting-fb5sw-i7-py3.10/lib/python3.10/site-packages/pydantic/_internal/_fields.py:161: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/root/.cache/pypoetry/virtualenvs/prompting-fb5sw-i7-py3.10/lib/python3.10/site-packages/pydantic/_internal/_fields.py:161: UserWarning: Field "model_manager" has conflict with protected namespace "model_".

You may be a

In [3]:
# model_manager = ModelManager(always_active_models=[ModelZoo.get_model_by_id(model_id=ModelZoo.models_configs[0].model_id)])

In [4]:
from pydantic import BaseModel

model_manager = ModelManager()
    
model_scheduler = AsyncModelScheduler(model_manager=model_manager)


In [14]:
import numpy as np

a = np.random.normal(size=10)
successes = np.sum([a < max_bound] and [a > min_bound])

In [5]:

# Start the scheduler asynchronously
asyncio.run(model_scheduler.start())


2024-08-14 12:20:32.275 | INFO     | __main__:run_scheduler:140 - Loading model casperhansen/llama-3-8b-instruct-awq for 10 seconds.


INFO 08-14 12:20:32 awq_marlin.py:77] The model is convertible to awq_marlin during runtime. Using awq_marlin kernel.
INFO 08-14 12:20:32 llm_engine.py:176] Initializing an LLM engine (v0.5.3.post1) with config: model='casperhansen/llama-3-8b-instruct-awq', speculative_config=None, tokenizer='casperhansen/llama-3-8b-instruct-awq', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=8000, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=awq_marlin, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None), seed=0, served_model_name=casperhansen/llama-3-8b-instruct-awq, use_v2_block_manager=False, en

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:00<00:00,  5.47it/s]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:01<00:00,  1.50it/s]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:01<00:00,  1.68it/s]



INFO 08-14 12:20:36 model_runner.py:692] Loading model weights took 5.3453 GB
INFO 08-14 12:20:38 gpu_executor.py:102] # GPU blocks: 16735, # CPU blocks: 2048
INFO 08-14 12:20:40 model_runner.py:980] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 08-14 12:20:40 model_runner.py:984] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 08-14 12:20:56 model_runner.py:1181] Graph capturing finished in 16 secs.


2024-08-14 12:20:56.461 | INFO     | __main__:load_model:88 - Model casperhansen/llama-3-8b-instruct-awq loaded. Current used RAM: 24.0 GB
2024-08-14 12:21:06.472 | INFO     | __main__:run_scheduler:153 - Unloading model casperhansen/llama-3-8b-instruct-awq after 10 seconds.
2024-08-14 12:21:06.933 | INFO     | __main__:run_scheduler:140 - Loading model casperhansen/mistral-nemo-instruct-2407-awq for 10 seconds.


...

INFO 08-14 12:35:17 awq_marlin.py:77] The model is convertible to awq_marlin during runtime. Using awq_marlin kernel.
INFO 08-14 12:35:17 llm_engine.py:176] Initializing an LLM engine (v0.5.3.post1) with config: model='casperhansen/mistral-nemo-instruct-2407-awq', speculative_config=None, tokenizer='casperhansen/mistral-nemo-instruct-2407-awq', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=8000, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=awq_marlin, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None), seed=0, served_model_name=casperhansen/mistral-nemo-instruct-2407-awq, use_v2_bl

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:00<00:00,  1.47it/s]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:01<00:00,  1.17it/s]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:01<00:00,  1.20it/s]



INFO 08-14 12:35:21 model_runner.py:692] Loading model weights took 7.8469 GB
INFO 08-14 12:35:23 gpu_executor.py:102] # GPU blocks: 12365, # CPU blocks: 1638
INFO 08-14 12:35:24 model_runner.py:980] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 08-14 12:35:24 model_runner.py:984] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 08-14 12:35:36 model_runner.py:1181] Graph capturing finished in 12 secs.


2024-08-14 12:35:36.736 | INFO     | __main__:load_model:88 - Model casperhansen/mistral-nemo-instruct-2407-awq loaded. Current used RAM: 24.0 GB


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 # Start the scheduler asynchronously                                                         │
│ ❱ 2 asyncio.run(model_scheduler.start())                                                         │
│   3                                                                                              │
│                                                                                                  │
│ /root/.cache/pypoetry/virtualenvs/prompting-fb5sw-i7-py3.10/lib/python3.10/site-packages/nest_as │
│ yncio.py:30 in run                                                                               │
│                                                                                                  │
│    27 │   │   loop.set_debug(debug)                                                              │
│    28 │   │   task = asyncio.ensure_future(main)                                                 │
│    29 │   │   try:                                                                               │
│ ❱  30 │   │   │   return loop.run_until_complete(task)                                           │
│    31 │   │   finally:                                                                           │
│    32 │   │   │   if not task.done():                                                            │
│    33 │   │   │   │   task.cancel()                                                              │
│                                                                                                  │
│ /root/.cache/pypoetry/virtualenvs/prompting-fb5sw-i7-py3.10/lib/python3.10/site-packages/nest_as │
│ yncio.py:92 in run_until_complete                                                                │
│                                                                                                  │
│    89 │   │   │   if f is not future:                                                            │
│    90 │   │   │   │   f._log_destroy_pending = False                                             │
│    91 │   │   │   while not f.done():                                                            │
│ ❱  92 │   │   │   │   self._run_once()                                                           │
│    93 │   │   │   │   if self._stopping:                                                         │
│    94 │   │   │   │   │   break                                                                  │
│    95 │   │   │   if not f.done():                                                               │
│                                                                                                  │
│ /root/.cache/pypoetry/virtualenvs/prompting-fb5sw-i7-py3.10/lib/python3.10/site-packages/nest_as │
│ yncio.py:115 in _run_once                                                                        │
│                                                                                                  │
│   112 │   │   │   else min(max(                                                                  │
│   113 │   │   │   │   scheduled[0]._when - self.time(), 0), 86400) if scheduled                  │
│   114 │   │   │   else None)                                                                     │
│ ❱ 115 │   │   event_list = self._selector.select(timeout)                                        │
│   116 │   │   self._process_events(event_list)                                                   │
│   117 │   │                                                                                      │
│   118 │   │   end_time = self.time() + self._clock_resolution                                    │
│                                                            

In [28]:
from vllm import SamplingParams

sampling_params = SamplingParams(seed=43, max_tokens=100, temperature=0.5, top_p=0.9, top_k=50)
list(model_manager.active_models.values())[0].generate("Hello, my name is", sampling_params=sampling_params)

Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.60s/it, est. speed input: 3.75 toks/s, output: 62.46 toks/s]


[RequestOutput(request_id=2, prompt='Hello, my name is', prompt_token_ids=[1, 22177, 1044, 2036, 2564, 1395], prompt_logprobs=None, outputs=[CompletionOutput(index=0, text=" [Your Name], and I am a [Your Age]-year-old [Your Gender] from [Your City/State/Country]. I am here to learn about [Your Interest or Goal]. I would like to ask you some questions about [Your Topic of Interest] if you don't mind.\n\n في: Hi, my name is [Your Name], and I am a [Your Age]-year-old [Your Gender] from [Your City/State/Country]. I am here to learn about", token_ids=(1766, 16994, 9878, 3605, 1321, 1362, 1855, 1261, 1766, 16994, 21191, 26118, 26098, 15962, 1766, 16994, 59944, 1093, 1562, 1766, 16994, 6308, 1047, 3906, 14787, 15543, 3077, 1362, 1855, 3226, 1317, 8178, 2314, 1766, 16994, 24789, 1505, 57148, 3077, 1362, 2168, 2479, 1317, 4237, 1636, 2269, 8352, 2314, 1766, 16994, 96620, 1307, 24789, 1093, 1693, 1636, 2607, 2405, 5759, 1338, 1819, 1058, 24665, 1044, 2036, 2564, 1395, 1766, 16994, 9878, 3605, 1

In [20]:
model_manager.load_model(ModelZoo.get_random())

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 model_manager.load_model(ModelZoo.get_random())                                              │
│   2                                                                                              │
│                                                                                                  │
│ in load_model:72                                                                                 │
│                                                                                                  │
│    69 │   │   │   │   │   break                                                                  │
│    70 │   │                                                                                      │
│    71 │   │   if self.used_ram + model_config.min_ram > self.total_ram or GPUInfo.free_memory    │
│ ❱  72 │   │   │   raise MemoryError(                                                             │
│    73 │   │   │   │   f"""Not enough RAM to load model {model_config.model_id}.                  │
│    74 │   │   │   │   │   Required: {model_config.min_ram} GB                                    │
│    75 │   │   │   │   │   Available in Model Manager: {self.total_ram - self.used_ram} GB        │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
MemoryError: Not enough RAM to load model casperhansen/mistral-nemo-instruct-2407-awq. 
                    Required: 24.0 GB
                    Available in Model Manager: 16.0 GB
                    Available in GPU: 4.622802734375 GB

In [8]:
import torch
free, total = torch.cuda.mem_get_info(0) 
free / (1024**3), total / (1024**3)

(42.999755859375, 44.3516845703125)

In [ ]:
del model_manager.active_models[model_config].llm_engine

In [ ]:
#llm is a vllm.LLM object
import gc
import torch
from vllm.model_executor.parallel_utils.parallel_state import destroy_model_parallel

destroy_model_parallel()
#del a vllm.executor.ray_gpu_executor.RayGPUExecutor object
del llm.llm_engine.model_executor
del llm
gc.collect()
torch.cuda.empty_cache()
import ray
ray.shutdown()

In [ ]:
import wandb

wandb.login(relogin=True, key="ae29a588c238d0e168d620e0b18a5e29e283935a")
wandb.init(
    project="validators",
    entity="felix-quinque-macrocosmos-ai",
)

In [ ]:
import wandb


WANDB = wandb.init(
        project="validator",
        entity="felix-quinque-macrocosmos"
    )

In [ ]:
import bittensor as bt

wallet = bt.wallet(name="dalkfjsl", hotkey=None)
metagraph = bt.metagraph(netuid=1)

In [ ]:
class Version:
    """Same as packaging.version, but also supports comparison to strings"""

    def __init__(self, version: str = "1.2.3"):
        self.version: str = version

    def __str__(self):
        return f"{self.version}"

    def __repr__(self):
        return f"{self.version}"

    def __eq__(self, other):
        other = other.version if isinstance(other, Version) else other
        return self.version == other

    def __le__(self, other):
        other = other.version if isinstance(other, Version) else other
        return True if all([v <= o for v, o in zip(self.version.split("."), other.split("."))]) else False

    def __lt__(self, other):
        other = other.version if isinstance(other, Version) else other
        return True if self <= other and self != other else False

    def __ge__(self, other):
        other = other.version if isinstance(other, Version) else other
        return True if not (self < other) else False

    def __gt__(self, other):
        other = other.version if isinstance(other, Version) else other
        return True if not (self <= other) else False



Version("1.2.3") < "1.2.2.4"

In [ ]:
version_split = "1.2.3".split(".")
(1000 * int(version_split[0])) + (10 * int(version_split[1])) + (1 * int(version_split[2]))

In [ ]:
import time
i = 0

start_time = time.time()
for _ in range(10):
    time.sleep(0.1)
    print(f"{-start_time+(start_time := time.time())} seconds have passed")
    # print(f"{(start_time := time.time())-start_time} seconds have passed")


In [ ]:
class Foo():
    def __init__(self):
        self.__bar = 42

foo = Foo()
foo.__dict__

In [ ]:
import bittensor as bt

In [ ]:
metagraph = bt.metagraph(netuid=61, network="test", sync=True, lite=False)

In [ ]:
metagraph.axons

In [ ]:
import numpy as np


x = np.reshape(np.random.random(size=10), (1,-1))
y = np.reshape(np.random.random(size=10), (-1,1))

float(x@y), x@y

In [ ]:
# from prompting.tasks.date_qa import DateQuestionAnsweringTask
from prompting.tasks.base_task import BaseTask
from prompting.rewards.reward import BaseRewardModel
from prompting.tasks.summarization import SummarizationTask, SummarizationRewardConfig
from prompting.tasks.qa import QuestionAnsweringTask, QARewardConfig

from prompting.datasets.wiki import WikiDataset
from prompting.datasets.base import BaseDataset
from pydantic import BaseModel, ConfigDict
import random
from typing import ClassVar
import bittensor as bt


class TaskConfig(BaseModel):
    task: BaseTask.__class__
    probability: float
    datasets: list[BaseDataset.__class__]
    reward_model: BaseRewardModel.__class__

    model_config = ConfigDict(arbitrary_types_allowed=True)


class TaskRegistry(BaseModel):
    tasks: ClassVar[list[TaskConfig]] = [
        TaskConfig(task=QuestionAnsweringTask, probability=0.6, datasets=[WikiDataset], reward_model=QARewardConfig),
        TaskConfig(
            task=SummarizationTask, probability=0.4, datasets=[WikiDataset], reward_model=SummarizationRewardConfig
        ),
        # TaskConfig(task=DateQuestionAnsweringTask, probability=0.2, datasets=[WikiDateDataset])
    ]

    @classmethod
    def random(self) -> TaskConfig:
        probabilities = [task.probability for task in self.tasks]
        selected_task = random.choices(self.tasks, probabilities)[0]
        return selected_task

    @classmethod
    def get_task_datasets(self, task: BaseTask.__class__):
        try:
            return [t.datasets for t in self.tasks if task is t.task][0]
        except Exception:
            bt.logging.error("Tried accessing non-registered task")
            return []

    @classmethod
    def get_random_task_dataset(self, task: BaseTask.__class__) -> BaseDataset.__class__:
        return random.choice(self.get_task_datasets(task))

    @classmethod
    def get_task_reward(self, task: BaseTask) -> BaseRewardModel:
        try:
            return [t.reward_model for t in self.tasks if task is t.task][0]
        except Exception:
            bt.logging.error("Tried accessing non-registered task")
            return []

    @classmethod
    def create_random_task(self, llm_pipeline) -> BaseTask:
        task_config = self.random()
        dataset = self.get_random_task_dataset(task_config.task)
        return task_config.task(
            llm_pipeline=llm_pipeline, context=dataset().next(), reward_config=task_config.reward_model()
        )


In [ ]:
TaskRegistry().get_task_reward(QuestionAnsweringTask)